# Stock Trading Bot

In [1]:
import pandas as pd 
import numpy as np 
import talib as ta 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score 
import alpaca_trade_api as tradeapi 
from alpaca_trade_api import TimeFrame, TimeFrameUnit 
from sklearn.ensemble import RandomForestClassifier
import time
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

# Api Communucation

In [2]:
SEC_KEY = '7mi6lnaTynfrzxfDYyYUarBpUJ35sSgVEVIj6RZG'
PUB_KEY ='PKDL8E9SBYH9GYYXXZNL'
BASE_URL ='https://paper-api.alpaca.markets'
symb = "AMZN"
api = tradeapi.REST(key_id= PUB_KEY, secret_key=SEC_KEY, base_url=BASE_URL) #For real trading, don't enter a base_url
position_Buy=None
position_Sell=None

# Class for Machine learning Accuracy and Prediction

In [3]:

def Algorithm(df):

        df['future_returns'] = df['close'].pct_change().shift(-1)
        signal=[]
        for i in range(len(df)):
                if (df['future_returns'][i]>0 ):
                    signal.append(1)
                elif (df['future_returns'][i]<0):
                    signal.append(2)
                else:
                    signal.append(3)
        df['signal']=signal
        df['Open-Close']=(df["open"]-df['close']).shift(-1)
        df['Close-High']=(df["close"]-df['high']).shift(-1)
        df['rsi'] = ta.RSI(df['close'].values, timeperiod=int(14))
        df['adx'] = ta.ADX(df['high'].values, df['low'].values,df['open'].values, timeperiod=int(50))
        df['sma'] = ta.SMA(df['close'],timeperiod=30)
        df['corr'] = df['close'].rolling(window=int(6.5*4)).corr(df['sma'])
        df['Close-Low']=(df["close"]-df['low']).shift(-1)

        df.dropna(inplace=True)
        y = df[['signal']].copy()
        X = df[[ 'rsi','adx', 'corr','volume','Open-Close','Close-Low']].copy()
        split_percentage = 0.8
        split = int(split_percentage*len(df))
        X_train = X[:split]
        y_train = y[:split]
        X_test =  X[split:]
        y_test =  y[split:]
        scaler=StandardScaler()
        X_train=scaler.fit_transform(X_train)
        X_test=scaler.fit_transform(X_test)
        # 
        # model=SVC()
        model = RandomForestClassifier(n_estimators=100,random_state=20)
        model.fit(X_train, y_train['signal'])
        y_pred = model.predict(X_test)
        Prediction=model.predict(X)[-1]
        accuracy =accuracy_score(y_test, y_pred)
        return accuracy,Prediction
    


In [4]:
while(True):
    print("Checking Price")
    try:
        df = api.get_barset(symb, TimeFrame(15, TimeFrameUnit.Minute), limit=1000).df  #Get one bar object for each of the past 5 minutes
    except Exception:
        print("Failed to Connect")
    df=df[symb]
    last_price = df.close[-1]                      
    accuracy,prediction=Algorithm(df)
    print("Last Price: " + str(last_price))
    print(f"Accuracy is {accuracy}")
    if accuracy >0.50 and prediction== 1 and position_Buy!="Buy" and position_Sell!="Sell": 
        print("Buy")
        api.submit_order(
            symbol=symb,
            qty=1,
            side='buy',
            type='market',
            time_in_force='gtc',)
        Buy=last_price
        position_Buy="Buy"
    elif accuracy >0.50 and prediction== 2 and position_Sell!="Sell" and position_Buy!="Buy": # If MA is more than 10cents above price, and we already bought
        print("Sell")
        api.submit_order(
            symbol=symb,
            qty=1,
            side='sell',
            type='market',
            time_in_force='gtc',)
        Sell=last_price
        position_Sell="Sell"
    elif position_Buy=="Buy" and (last_price>=Buy+3 or last_price<=Buy-3):
         print("Buy Closed")
         api.submit_order(
            symbol=symb,
            qty=1,
            side='sell',
            type='market',
            time_in_force='gtc')
         position_Buy=None
    elif position_Sell=="Sell" and (last_price>=Sell+3 or last_price<=Sell-3):
         print("Sell Closed")
         api.submit_order(
            symbol=symb,
            qty=1,
            side='buy',
            type='market',
            time_in_force='gtc')
         position_Sell=None
    time.sleep(60)

Checking Price


C:\Users\Azaz\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:438: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


Last Price: 2951.15
Accuracy is 0.8555555555555555
Buy
Checking Price


C:\Users\Azaz\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:438: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


Last Price: 2947.495
Accuracy is 0.8333333333333334
Buy Closed
Checking Price


...

Last Price: 2942.99
Accuracy is 0.8388888888888889
Checking Price


C:\Users\Azaz\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:438: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


Last Price: 2942.44
Accuracy is 0.8388888888888889
Checking Price


C:\Users\Azaz\AppData\Roaming\Python\Python39\site-packages\sklearn\base.py:438: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


Last Price: 2942.44
Accuracy is 0.8388888888888889
